# Lab 13: Path Planning and Execution

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import time
import pathlib
import os
from utils import get_max, load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

from controller import *

import nest_asyncio
nest_asyncio.apply()

2022-05-18 16:52:05,440 | INFO     |: Logger demo_notebook.log initialized.


In [2]:
# Start the plotter
START_PLOTTER()

2022-05-18 16:52:14.886 Python[93043:4148537] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/cb/bssy6wvs7yx5fqv9pd5x930w0000gn/T/org.python.python.savedState


# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [3]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        return get_max(loc.bel_bar)

    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        rc.spin_360(80)
        
        while rc.sensor_readings is None or len(rc.sensor_readings) <= 18:
            asyncio.run(asyncio.sleep(0.05))
        rc.stop_recording()
        
        # Collect & display the full sensor data
        cols = [
            "time", "dist_r", "dist_f", "acc_x", "acc_y", "acc_z", "gyr_x",
            "gyr_y", "gyr_z", "mag_x", "mag_y", "mag_z", "temp"
        ]
        sensor_data = pd.DataFrame(rc.sensor_arr, columns=cols)
        
        sensor_ranges = (np.flip(sensor_data["dist_f"][:18].to_numpy()))[np.newaxis].T
        sensor_ranges /= 1000
        print(sensor_ranges)
        
        return sensor_ranges, sensor_ranges


In [3]:
START_PLOTTER()

2022-05-18 16:27:42.487 Python[92352:4126078] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/cb/bssy6wvs7yx5fqv9pd5x930w0000gn/T/org.python.python.savedState


In [4]:
STOP_PLOTTER()

2022-05-18 14:21:50,840 | INFO     |: Plotter is stopped


In [5]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2022-05-18 16:52:37,529 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:83:A5:6D:02:3C
2022-05-18 16:52:41,936 | INFO     |: Connected to C0:83:A5:6D:02:3C


In [6]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2022-05-18 16:52:43,200 | INFO     |:  | Number of observations per grid cell: 18
2022-05-18 16:52:43,211 | INFO     |:  | Precaching Views...


/Users/trueshot/Documents/git/fast-robots/lab13/ECE4960-sim-release/localization.py:144: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[


2022-05-18 16:52:48,067 | INFO     |:  | Precaching Time: 4.840 secs
2022-05-18 16:52:48,068 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-18 16:52:48,069 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [8]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Initialize robot controller logic
rc = RobotControl(ble)

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2022-05-16 16:29:56,554 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-16 16:29:56,560 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[1.299]
 [1.304]
 [1.335]
 [1.327]
 [1.242]
 [1.308]
 [1.311]
 [1.279]
 [1.358]
 [1.276]
 [1.284]
 [1.301]
 [1.273]
 [1.278]
 [1.346]
 [1.31 ]
 [1.11 ]
 [1.167]]
2022-05-16 16:30:07,340 | INFO     |: Update Step
2022-05-16 16:30:07,346 | INFO     |:      | Update Time: 0.004 secs
2022-05-16 16:30:07,347 | INFO     |: Bel index     : (5, 4, 10) with prob = 0.8480107
2022-05-16 16:30:07,348 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2022-05-16 16:30:07,350 | INFO     |: Belief        : (0.000, 0.000, 30.000)


In [7]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Initialize robot controller logic
rc = RobotControl(ble)

2022-05-18 16:52:49,429 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-18 16:52:49,430 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


In [8]:
# Lab 13 waypoints (in grid cell units)
waypoints_in_ft = [
    # (-4, -3, -135),
    (-2, -1, -180),
    (1, -1, -180),
    (2, -3, 120),
    (5, -3, -180),
    (5, -2, -90),
    (5, 3, -90),
    (0, 3, 0),
    (0, 0, 90),
]

waypoints_in_m = []
for wp in waypoints_in_ft:
    waypoint = (wp[0] * 0.308, wp[1] * 0.308, wp[2])
    waypoints_in_m.append(waypoint)

waypoints = []
for wp in waypoints_in_m:
    waypoints.append(loc.mapper.to_map(*wp))
print(waypoints)

# Helper functions
def rot_delta_to_pwm(angle, boost=0):
    """
    Returns:
        speed (PWM)
        time (sec)
    """
    speed = 100 + boost * 2
    time = (0.19 * abs(angle) / speed) ** 0.5
    return speed, time if angle > 0 else -time * 0.95

def trans_delta_to_pwm(dist, boost=0):
    """
    Returns:
        speed (PWM)
        time (sec)
    """
    speed = 60 + boost * 2
    time = (1.33 * dist) ** 0.6
    return speed, time

def turn_left_90():
    rc.rotate_left(97)
    time.sleep(0.5)
    rc.active_stop()

def turn_right_90():
    rc.rotate_right(103)
    time.sleep(0.5)
    rc.active_stop()

# Execute a forward trajectory for the robot. If there are any 
# obstacles in the way, run obstacle avoidance algorithm and 
# signal to the calling function that obstacles were in the way.
def execute_safe_trajectory(speed_trans, time_trans):
    print(f"Moving forward: {speed_trans} PWM")
    rc.move_forward(speed_trans)

    now = time.time()
    while time.time() - now < time_trans:
        if decode_sensor_data(rc.latest_sensor_reading).distF < 300:
            rc.active_stop()
            avoid_obstacles()
            return True
    rc.active_stop()

    return False
    
async def avoid_obstacles():
    print("Avoiding obstacles")
    turn_left_90()
    
    while decode_sensor_data(rc.latest_sensor_reading).distR < 300:
        if decode_sensor_data(rc.latest_sensor_reading).distF < 300:
            avoid_obstacles()
        else:
            rc.move_forward(60)
            await asyncio.sleep(0.2)
            rc.stop()
    await asyncio.sleep(0.1)
    turn_right_90()
    
    # Get Observation Data by executing a 360 degree rotation motion
#     loc.get_observation_data()

#     # Run Update Step
#     loc.update_step()
#     loc.plot_update_step_data(plot_data=True)

[(3, 2, 0), (7, 2, 0), (8, 0, 15), (11, 0, 0), (11, 1, 4), (11, 7, 4), (6, 7, 9), (6, 4, 13)]


In [9]:
# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)
argmax_bel = get_max(loc.bel)
current_belief = loc.mapper.from_map(*argmax_bel[0])

[[1.166]
 [1.043]
 [1.073]
 [0.598]
 [0.581]
 [0.441]
 [0.376]
 [0.372]
 [0.458]
 [0.508]
 [0.401]
 [0.382]
 [0.439]
 [0.625]
 [1.071]
 [3.554]
 [1.986]
 [0.825]]
2022-05-18 16:54:30,660 | INFO     |: Update Step
2022-05-18 16:54:30,665 | INFO     |:      | Update Time: 0.003 secs
2022-05-18 16:54:30,666 | INFO     |: Bel index     : (2, 0, 14) with prob = 0.9999999
2022-05-18 16:54:30,667 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2022-05-18 16:54:30,668 | INFO     |: Belief        : (-0.914, -1.219, 110.000)


In [ ]:
# Run through each motion steps
i = 0
boost = 0
while True:
    print("\n\n----------------WP", i, "----------------")
        
    # Prediction Step
    current_odom = waypoints[i]
    prev_odom = waypoints[0] if i == 0 else waypoints[i - 1]
    loc.prediction_step(current_odom, prev_odom)
    # loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()

    # Run Update Step
    loc.update_step()
    loc.plot_update_step_data(plot_data=True)
    argmax_bel = get_max(loc.bel)
    current_belief = loc.mapper.from_map(*argmax_bel[0])
    print(f"Current belief: {current_belief}")
    
    # Check if we reached a waypoint
    x, y, _ = argmax_bel[0]
    way_x = waypoints[i][0]
    way_y = waypoints[i][1]
    if way_x - 1 <= x <= way_x + 1 and way_y - 1 <= y <= way_y + 1:
        i += 1
        print(f"New goal waypoint {i}")
    
    # Termination condition
    if i == len(waypoints):
        print("We done!")
        break
    
    # Compute the necessary control to get to the next waypoint (assuming no obstacles)
    delta_rot_1, delta_trans, delta_rot_2 = loc.compute_control(waypoints_in_m[i], current_belief)
    print(f"Control values: {delta_rot_1}, {delta_trans}, {delta_rot_2}")
    speed_rot_1, time_rot_1 = rot_delta_to_pwm(delta_rot_1, boost)
    speed_trans, time_trans = trans_delta_to_pwm(delta_trans)
    speed_rot_2, time_rot_2 = rot_delta_to_pwm(delta_rot_2, boost)
    
    # Invoke motor inputs...
    # First rotation:
    if time_rot_1 < 0:
        print(f"Rotating right {speed_rot_1}")
        rc.rotate_right(speed_rot_1)
    else:
        print(f"Rotating left {speed_rot_1}")
        rc.rotate_left(speed_rot_1)
    await asyncio.sleep(abs(time_rot_1))
    rc.active_stop()
    
    await asyncio.sleep(1)
    
    # Try to move forward with the forward controls.
    # If there are obstacles, scrap the rest of the control and try again with new localization data.
    obstacles_avoided = execute_safe_trajectory(speed_trans, time_trans)
    if obstacles_avoided:
        continue
    
    await asyncio.sleep(1)
    
    # Otherwise... continue with the second rotation
    if time_rot_2 < 0:
        print(f"Rotating right {speed_rot_2}")
        rc.rotate_right(speed_rot_2)
    else:
        print(f"Rotating left {speed_rot_2}")
        rc.rotate_left(speed_rot_2)
    await asyncio.sleep(abs(time_rot_2))
    rc.active_stop()
    
    boost += 1


# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------WP 0 ----------------
2022-05-18 16:54:35,421 | INFO     |: Prediction Step
2022-05-18 16:54:35,504 | INFO     |:  | Prediction Time: 0.079 secs
[[2.793]
 [1.109]
 [1.046]
 [1.065]
 [1.089]
 [0.586]
 [0.439]
 [0.4  ]
 [0.428]
 [0.554]
 [0.451]
 [0.382]
 [0.389]
 [0.494]
 [0.785]
 [1.02 ]
 [3.561]
 [1.892]]
2022-05-18 16:54:46,917 | INFO     |: Update Step
2022-05-18 16:54:46,921 | INFO     |:      | Update Time: 0.003 secs
2022-05-18 16:54:46,922 | INFO     |: Bel index     : (1, 1, 11) with prob = 1.0
2022-05-18 16:54:46,923 | INFO     |: Bel_bar prob at index = 7.315639963206552e-09
2022-05-18 16:54:46,924 | INFO     |: Belief        : (-1.219, -0.914, 50.000)
Current belief: (-1.2191999999999998, -0.9144, 50.0)
Control values: -4.848424217331683, 0.8553193555625874, 134.84842421733168
Rotating right 100
Moving forward: 60 PWM
Rotating left 100
-------------------------------------


----------------WP 0 ----------------
2022-05-18 16:54:51,672 | INFO     |: Predict

/var/folders/cb/bssy6wvs7yx5fqv9pd5x930w0000gn/T/ipykernel_93025/3360257187.py:66: RuntimeWarning: coroutine 'avoid_obstacles' was never awaited
  avoid_obstacles()


[[3.703]
 [3.013]
 [1.297]
 [1.356]
 [1.492]
 [1.284]
 [1.177]
 [1.169]
 [0.818]
 [0.441]
 [0.35 ]
 [0.317]
 [0.332]
 [0.383]
 [0.304]
 [0.292]
 [0.227]
 [0.225]]
2022-05-18 16:55:13,891 | INFO     |: Update Step
2022-05-18 16:55:13,894 | INFO     |:      | Update Time: 0.003 secs
2022-05-18 16:55:13,895 | INFO     |: Bel index     : (9, 3, 17) with prob = 0.9999999
2022-05-18 16:55:13,896 | INFO     |: Bel_bar prob at index = 1.8522359652750974e-34
2022-05-18 16:55:13,897 | INFO     |: Belief        : (1.219, -0.305, 170.000)
Current belief: (1.2192000000000005, -0.30479999999999985, 170.0)
Control values: 10.099905355615533, 1.8352027898845407, -0.09990535561553315
Rotating left 102
Moving forward: 60 PWM
Rotating right 102
-------------------------------------


----------------WP 0 ----------------
2022-05-18 16:55:18,807 | INFO     |: Prediction Step
2022-05-18 16:55:18,852 | INFO     |:  | Prediction Time: 0.044 secs
[[1.179]
 [0.814]
 [0.688]
 [0.633]
 [0.927]
 [1.835]
 [2.311]


/var/folders/cb/bssy6wvs7yx5fqv9pd5x930w0000gn/T/ipykernel_93025/3360257187.py:66: RuntimeWarning: coroutine 'avoid_obstacles' was never awaited
  avoid_obstacles()


[[0.756]
 [0.765]
 [0.844]
 [1.18 ]
 [1.823]
 [1.919]
 [1.745]
 [2.766]
 [3.149]
 [0.067]
 [0.037]
 [0.013]
 [0.017]
 [0.038]
 [0.063]
 [0.106]
 [0.182]
 [0.191]]
2022-05-18 16:57:25,863 | INFO     |: Update Step
2022-05-18 16:57:25,867 | INFO     |:      | Update Time: 0.004 secs
2022-05-18 16:57:25,868 | INFO     |: Bel index     : (1, 4, 1) with prob = 0.5081536
2022-05-18 16:57:25,869 | INFO     |: Bel_bar prob at index = 1.5386720452144e-07
2022-05-18 16:57:25,870 | INFO     |: Belief        : (-1.219, 0.000, -150.000)
Current belief: (-1.2191999999999998, 1.3877787807814457e-16, -150.0)
Control values: 123.27532317423845, 2.0546861171478237, 146.72467682576155
Rotating left 118
Moving forward: 60 PWM
Rotating left 118
-------------------------------------


----------------WP 2 ----------------
2022-05-18 16:57:31,693 | INFO     |: Prediction Step
2022-05-18 16:57:31,787 | INFO     |:  | Prediction Time: 0.093 secs
[[0.598]
 [0.608]
 [0.311]
 [0.195]
 [0.137]
 [0.115]
 [0.103]
 [

/var/folders/cb/bssy6wvs7yx5fqv9pd5x930w0000gn/T/ipykernel_93025/3360257187.py:66: RuntimeWarning: coroutine 'avoid_obstacles' was never awaited
  avoid_obstacles()


[[2.291]
 [2.263]
 [3.355]
 [1.416]
 [1.305]
 [3.085]
 [2.222]
 [1.297]
 [0.989]
 [0.898]
 [0.823]
 [0.301]
 [0.178]
 [0.122]
 [0.102]
 [0.098]
 [0.103]
 [0.101]]
2022-05-18 16:58:24,544 | INFO     |: Update Step
2022-05-18 16:58:24,574 | INFO     |:      | Update Time: 0.017 secs
2022-05-18 16:58:24,578 | INFO     |: Bel index     : (10, 8, 17) with prob = 0.9999999
2022-05-18 16:58:24,580 | INFO     |: Bel_bar prob at index = 0.0005848466707414098
2022-05-18 16:58:24,582 | INFO     |: Belief        : (1.524, 1.219, 170.000)
Current belief: (1.5240000000000002, 1.2192000000000003, 170.0)
Control values: 77.03930015451704, 2.3276104141372116, -127.03930015451704
Rotating left 124
Moving forward: 60 PWM
Rotating right 124
-------------------------------------


----------------WP 2 ----------------
2022-05-18 16:58:30,523 | INFO     |: Prediction Step
2022-05-18 16:58:30,615 | INFO     |:  | Prediction Time: 0.091 secs
2022-05-18 16:58:32,910 | INFO     |: Disconnected from B45C71B2-3BC

In [10]:
current_belief

(-0.9143999999999998, -1.2191999999999998, 90.0)

In [19]:
# Check if we reached a waypoint
x, y, _ = argmax_bel[0]
way_x = waypoints[0][0]
way_y = waypoints[0][1]
print(x,y)
x, y = 3,2
print(way_x, way_y)
if way_x - 2 <= x <= way_x + 2 and way_y - 2 <= y <= way_y + 2:
    print("True")
#     i += 1
#     print(f"New goal waypoint {i}")

2 0
3 2
True


In [18]:
decode_sensor_data(rc.latest_sensor_reading).distF

1242.0

In [44]:
execute_safe_trajectory(*trans_delta_to_pwm(1))

Moving forward: 60 PWM


False

2022-05-17 16:52:34,942 | INFO     |: Disconnected from B45C71B2-3BCA-639D-7AF3-3C14C2003C3E


In [14]:
turn_left_90()

In [15]:
turn_right_90()

In [14]:
rc.active_stop()

In [8]:
speed, t = rot_delta_to_pwm(-55.36533691396145)
if t < 0:
    print(f"Rotating right {speed}")
    rc.rotate_right(speed)
else:
    print(f"Rotating left {speed}")
    rc.rotate_left(speed)
await asyncio.sleep(abs(t))
rc.active_stop()

Rotating right 100


In [15]:
time_rot_1

-0.3081196382465996

In [18]:
delta_rot_1, delta_trans, delta_rot_2 = (-24.848424217331683, 0.8553193555625874, 134.84842421733168)
speed_rot_1, time_rot_1 = rot_delta_to_pwm(delta_rot_1)
speed_trans, time_trans = trans_delta_to_pwm(delta_trans)
speed_rot_2, time_rot_2 = rot_delta_to_pwm(delta_rot_2)

# Invoke motor inputs...
# First rotation:
if time_rot_1 < 0:
    print(f"Rotating right {speed_rot_1}")
    rc.rotate_right(speed_rot_1)
else:
    print(f"Rotating left {speed_rot_1}")
    rc.rotate_left(speed_rot_1)
time.sleep(abs(time_rot_1))
rc.active_stop()

# Try to move forward with the forward controls.
# If there are obstacles, scrap the rest of the control and try again with new localization data.
obstacles_avoided = execute_safe_trajectory(speed_trans, time_trans)
if obstacles_avoided:
    return

# Otherwise... continue with the second rotation
if time_rot_2 < 0:
    print(f"Rotating right {speed_rot_2}")
    rc.rotate_right(speed_rot_2)
else:
    print(f"Rotating left {speed_rot_2}")
    rc.rotate_left(speed_rot_2)
time.sleep(abs(time_rot_2))
rc.active_stop()


Rotating right 100
Moving forward: 60 PWM


SyntaxError: 'return' outside function (1893023247.py, line 21)